# Using Keras_Tuner to tune the hyper parameters of DL models
---

In [1]:
# imports
import tensorflow as tf
print(tf.__version__)

2.2.0-rc3


In [6]:
# install keras tuner
!pip install -U keras-tuner

     |████████████████████████████████| 61kB 3.0MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=44968692e005610c1c6f8b939a5aaa28e700554472a0a77cc60bd7a6c99efbbe
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=3401f94aaa896bf5e1ba29321b5ecd62de23c007ab3dc9ce7e1a820e79f0a6bd
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [2]:
# download the fashion MNIST data
fashion_mnist = tf.keras.datasets.fashion_mnist
(train_images, train_label), (test_images, test_labels) = fashion_mnist.load_data()

# rescale images betwee 0-1
train_images = train_images/255
test_images = test_images/255

4423680/4422102 [==============================] - 0s 0us/step


In [3]:
print(train_images.shape, len(train_images), test_images.shape, len(test_images))

# reshape the images to 28,28,1
train_images = train_images.reshape(len(train_images), 28, 28, 1)
test_images = test_images.reshape(len(test_images), 28, 28, 1)


(60000, 28, 28) 60000 (10000, 28, 28) 10000


In [0]:
# build a CNN model using keras tuner

def build_model(hp):
  model = tf.keras.models.Sequential([
      # first conv layer
      tf.keras.layers.Conv2D(
          filters=hp.Int('conv1_filter', min_value=32, max_value=128, step=16),
          kernel_size=hp.Choice('conv1_kernel', values=[3,5]),
          activation='relu',
          input_shape=(28, 28, 1)),

      # second conv layer
      tf.keras.layers.Conv2D(
          filters=hp.Int('conv2_filter', min_value=32, max_value=64, step=16),
          kernel_size=hp.Choice('conv2_kernel', values=[3,5]),
          activation='relu'),

      # flatten the output
      tf.keras.layers.Flatten(),

      # Dense layer
      tf.keras.layers.Dense(
          units=hp.Int("Dense1_units", min_value=32, max_value=128, step=16),
          activation='relu'),
      
      # softmax layer
      tf.keras.layers.Dense(10, activation='softmax')])
  
  model.compile(
      optimizer=tf.keras.optimizers.Adam(learning_rate=hp.Choice("Alearning_rate", [1e-4, 1e-3, 1e-2, 1e-1, 3e-1])),
      loss='sparse_categorical_crossentropy',
      metrics=['accuracy'])

  return model
  

In [19]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

# create a random_search object to do trials
tune = RandomSearch(hypermodel=build_model, objective='val_accuracy', max_trials=5, 
                    project_name='fashion MNIST', directory='output')

INFO:tensorflow:Reloading Oracle from existing project output/fashion MNIST/oracle.json
INFO:tensorflow:Reloading Tuner from output/fashion MNIST/tuner0.json


In [20]:
# begin the trials
tune.search(train_images, train_label, epoch=3, validation_split=0.2)

INFO:tensorflow:Oracle triggered exit


In [21]:
tuned_model = tune.get_best_models(num_models=1)[0]
tuned_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 112)       1120      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 48)        48432     
_________________________________________________________________
flatten (Flatten)            (None, 27648)             0         
_________________________________________________________________
dense (Dense)                (None, 144)               3981456   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1450      
Total params: 4,032,458
Trainable params: 4,032,458
Non-trainable params: 0
_________________________________________________________________


In [22]:
# we will use the best tuned model to train now
history = tuned_model.fit(train_images, train_label, epochs=10, validation_split=0.1)

Epoch 1/10
1688/1688 [==============================] - 8s 4ms/step - loss: 0.2392 - accuracy: 0.9111 - val_loss: 0.2601 - val_accuracy: 0.9053
Epoch 2/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1694 - accuracy: 0.9381 - val_loss: 0.2467 - val_accuracy: 0.9115
Epoch 3/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1131 - accuracy: 0.9580 - val_loss: 0.2488 - val_accuracy: 0.9133
Epoch 4/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0754 - accuracy: 0.9725 - val_loss: 0.3277 - val_accuracy: 0.9163
Epoch 5/10
1688/1688 [==============================] - 8s 4ms/step - loss: 0.0514 - accuracy: 0.9811 - val_loss: 0.3799 - val_accuracy: 0.9100
Epoch 6/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0357 - accuracy: 0.9879 - val_loss: 0.4434 - val_accuracy: 0.9090
Epoch 7/10
1688/1688 [==============================] - 8s 4ms/step - loss: 0.0277 - accuracy: 0.9902 - val_loss: 0.4384 - val_accuracy:

In [23]:
train_acc = tuned_model.evaluate(train_images, train_label)
test_acc = tuned_model.evaluate(test_images, test_labels)

313/313 [==============================] - 1s 3ms/step - loss: 0.6337 - accuracy: 0.9089
